# Galaxy cluster weak gravitational lensing analysis

<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px" alt="Rubin Observatory logo, a graphical representation of turning stars into data.">
<br>
Contact author(s): Shenming Fu <br>
Last verified to run: 2025-02-xx <br>
LSST Science Pipelines version: Weekly 2024_50 <br>
Container Size: medium <br>
Targeted learning level: beginner <br>

In [ ]:
%load_ext pycodestyle_magic
%flake8_on
import logging
logging.getLogger("flake8").setLevel(logging.FATAL)

**Description:** This notebook demonstrates how to DP0 data for galaxy cluster weak gravitational lensing analysis.

**Skills:** Select background galaxies by color and make a mean shape profile.

**LSST Data Products:** Images (deepCoadd) and catalogs (object table).

**Packages:** lsst.daf.butler, lsst.rsp

**Credit:** This tutorial was originally developed by Shenming Fu.

**Get Support:**
Find DP0-related documentation and resources at <a href="https://dp0.lsst.io">dp0.lsst.io</a>. Questions are welcome as new topics in the <a href="https://community.lsst.org/c/support/dp0">Support - Data Preview 0 Category</a> of the Rubin Community Forum. Rubin staff will respond to all questions posted there.

## 1. Introduction

Galaxy clusters are the largest gravitionally bound objects in the Universe. The large mass of a cluster coherently distorts the images of backgroud galaxies over a large sky area via weak gravitational lensing (WL). 
This notebook gives an example of detecting lensing signal of clusters using DP0 data.

### 1.1. Import packages
Import general packages, the LSST Science Pipelines packages for bulter and display, and Rubin Science Platform TAP package. 

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('tableau-colorblind10')

import numpy as np
from scipy.stats import binned_statistic, bootstrap
from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

from lsst.rsp import get_tap_service

## 2. Get the coadd object catalog around a cluster
There is a known cluster at (55.749, -32.273) for RA, DEC. Make a cone search for primary objects with radius 15 arcmin using TAP. 
The details of TAP can be found at the notebook 02a. 

Select galaxies by extendedness (difference between the PSF photometry and the CModel photometry); see details [here](https://github.com/rubin-dp0/delegate-contributions-dp02/tree/main/extendedness). Use the g and r-band color information to remove foreground galaxies, and include some related flag cuts. Also, use r-band for shape analysis (common in WL analysis), and include some quality cuts for the shape measurements ([Mandelbaum+2018](https://ui.adsabs.harvard.edu/abs/2018PASJ...70S..25M/abstract)). 

For simplicity, skip the use of photometric redshift in this notebook.

Finally, save the results into an astropy table.

In [ ]:
service = get_tap_service("tap")
assert service is not None

In [ ]:
center_ra = 55.749
center_dec = -32.273
radius = 15/60.

str_center_coords = str(center_ra) + ", " + str(center_dec)
str_radius = str(radius)

query = "SELECT r_ra, r_decl, "\
        "r_cModelFlux, r_cModelFluxErr, g_cModelFlux, g_cModelFluxErr, "\
        "r_hsmShapeRegauss_e1, r_hsmShapeRegauss_e2, x, y "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), "\
        "CIRCLE('ICRS', " + str_center_coords + ", " + str_radius + ")) = 1 "\
        "AND detect_isPrimary = 1 "\
        "AND r_extendedness = 1 "\
        "AND r_extendedness_flag = 0 "\
        "AND r_cModel_flag = 0 "\
        "AND g_cModel_flag = 0 "\
        "AND r_psfFlux_flag = 0 "\
        "AND r_centroid_flag = 0 "\
        "AND r_hsmShapeRegauss_sigma < 0.4 "\
        "AND r_blendedness < 0.42 "


print(query)

job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)
assert job.phase == 'COMPLETED'

results = job.fetch_result().to_table().to_pandas()
print(results)

job.delete()

data = Table.from_pandas(results)

del query, results

Apply some further cuts on the g and r-band photometry to ensure the quality. Also, require the measured ellipticity to be within 2; this removes galaxies with very large measured ellipticities that are unphysical, but allows galaxies with measured ellipticities slightly larger than 1 ([Mandelbaum+2018](https://ui.adsabs.harvard.edu/abs/2018PASJ...70S..25M/abstract)). For demonstration, skip shear calibration but just study the mean galaxy shape in this notebook.

In [ ]:
data = data[data['r_cModelFluxErr'] > 0]
data = data[data['g_cModelFluxErr'] > 0]

r_SNR = data['r_cModelFlux'] / data['r_cModelFluxErr']
g_SNR = data['g_cModelFlux'] / data['g_cModelFluxErr']
e1 = data['r_hsmShapeRegauss_e1']
e2 = data['r_hsmShapeRegauss_e2']

sel = r_SNR > 5.
sel &= g_SNR > 5.

sel &= (e1**2 + e2**2)**0.5 < 2.

data_s = data[sel]

In [ ]:
del data

## 3. Select galaxies by color

Create the g-r vs r color-magnitude diagram (CMD) of galaxies. Clusters generally have a typical red sequence (RS) in CMD due to evolution -- those red galaxies are the oldest and reddest in the cluster. Galaxies that are redder than RS are background galaxies. Thus, use colors to select a sample of background galaxies, and a sample of bright cluster member galaxies -- they show different colors in CMD. See also examples [here](https://dp0-2.lsst.io/tutorials-examples/portal-4.html).

First, convert flux into magnitude (notebook 01) and compute the magnitude uncertainty. 

In [ ]:
r_cModel = -2.5*np.log10(data_s['r_cModelFlux']) + 31.4
g_cModel = -2.5*np.log10(data_s['g_cModelFlux']) + 31.4

fac = 2.5 / np.log(10)
r_cModel_err = fac / r_SNR[sel]

Make CMD.

In [ ]:
plt.scatter(r_cModel,
            g_cModel - r_cModel,
            s=0.03, c='k', marker='.')

sel_rs = g_cModel - r_cModel > 1.0
sel_rs &= g_cModel - r_cModel < 1.2
sel_rs &= r_cModel < 23.

plt.scatter(r_cModel[sel_rs],
            (g_cModel - r_cModel)[sel_rs],
            s=3, c='r', alpha=0.3,
            label='RS', marker='^')

sel_bg = g_cModel - r_cModel > 1.35

plt.scatter(r_cModel[sel_bg],
            (g_cModel - r_cModel)[sel_bg],
            s=3, c='b', alpha=0.3,
            label='background', marker='s')

plt.legend()

plt.xlim(15, 27)
plt.ylim(-1, 3.)
plt.xlabel('r')
plt.ylabel('g - r')
plt.minorticks_on()
plt.savefig('RS.png')

> Figure 1: Color-magnitude of galaxies with markers showing the selected background galaxies and the bright red sequence galaxies. 

Check the relation between the r-band magnitude and uncertainty, and compare that with SNR. 

In [ ]:
plt.figure()

plt.scatter(r_cModel,
            r_cModel_err,
            s=0.1, c='k')

ls_list = ['-', '--', '-.']
for i, SNR in enumerate([5, 10, 20]):
    plt.axhline(fac / SNR, c='C%d'%i, label='SNR=%d'%SNR, ls=ls_list[i])

plt.xlim(16, 28)
plt.ylim(0, 0.25)

plt.axvline(26, c='y', ls=':', label='mag=26')

plt.xlabel('r')
plt.ylabel('r_err')

plt.legend()

> Figure 2: r-band magnitude and uncertainty of galaxies.

Now, check the distribution of galaxies on the sky. Clearly, the RS galaxies have a concentration near the cluster center, but the background ones are uniformally distributed. 

In [ ]:
ra = data_s['r_ra']
dec = data_s['r_decl']

plt.figure(figsize=(5, 5))
plt.scatter(ra[sel_rs], dec[sel_rs], s=3, ec='r',
            label="RS", marker='^')
plt.scatter(ra[sel_bg], dec[sel_bg], s=1, ec='b',
            label="background", marker='s')
plt.xlabel('RA [deg]')
plt.ylabel('DEC [deg]')
plt.gca().invert_xaxis()
plt.legend()

plt.savefig('RS_sel.png')

> Figure 3: Sky distribution of the selected background and RS galaxies.

Also check the coadd image and see how the RS galaxies look like -- those bright large cluster galaxies do get selected. 

In [ ]:
config = 'dp02'
collections = '2.2i/runs/DP0.2'
butler = dafButler.Butler(config, collections=collections)

datasetType = 'deepCoadd'
dataId = {'tract': 4431, 'patch': 17, 'band': 'r'}
coadd = butler.get(datasetType, dataId=dataId)

In [ ]:
x = data_s['x']
y = data_s['y']
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(coadd.image)
plt.scatter(x[sel_rs], y[sel_rs], ec='r', fc='none')
plt.xlim(12000, 16000)
plt.ylim(8000, 12000)
plt.xlabel('x [pix]')
plt.ylabel('y [pix]')
plt.savefig('RS_sel.png')


> Figure 4: Coadd r-band image with the selected RS galaxies.

In [ ]:
del coadd

## 4. Lensing analysis

Compute the tangential and cross ellipticities of background galaxies toward the cluster center. Also, compute the radial distance in arcmin. 

In [ ]:
e1 = data_s['r_hsmShapeRegauss_e1']
e2 = data_s['r_hsmShapeRegauss_e2']

coord0 = SkyCoord(center_ra, center_dec, frame='icrs', unit='deg')
coord1 = SkyCoord(ra, dec, frame='icrs', unit='deg')

position_angle = coord0.position_angle(coord1).rad + np.pi/2.

e_t = - e1 * np.cos(2.*position_angle) - e2 * np.sin(2.*position_angle)
e_x = + e1 * np.sin(2.*position_angle) - e2 * np.cos(2.*position_angle)

r = coord0.separation(coord1).arcmin

Check the distributions of the tangential ellipticities and the cross ellipticities. They show Gaussian-like distributions, but the tangential one has a mean above zero (weakly), while the cross one has a mean close to zero.

In [ ]:
bins = np.linspace(-1, 1, 30)
mid = 0.5 * (bins[1:] + bins[:-1])

plt.figure()

n_e_t, _, im = plt.hist(e_t[sel_bg], bins=bins, histtype='step',
                        density=True, label='e_t')
n_e_x, _, im = plt.hist(e_x[sel_bg], bins=bins, histtype='step',
                        density=True, ls='--', label='e_x')

plt.axvline(np.mean(e_t[sel_bg]), c='C0', label='mean[e_t]')
plt.axvline(np.mean(e_x[sel_bg]), c='C1', ls='--', label='mean[e_x]')

plt.xlabel('ellipticity')
plt.ylabel('count')

plt.legend()

> Figure 5: Distributions of the tangential ellipticities and the cross ellipticities of the selected background galaxies.

The difference between two distributions shows a dipole feature as expected. 

In [ ]:
plt.figure()
plt.step(mid, n_e_t - n_e_x, where='mid')
plt.axvline(0, c='k', ls=':')
plt.xlabel('ellipticity')
plt.ylabel('count difference')

> Figure 6: Difference between the distributions of the tangential ellipticities and the cross ellipticities.

Bin the data by radial distance to find the mean. Estimate the error bar by std/sqrt(N). 
When plotting the cross component result, shift the data point slightly for visualization.

In [ ]:
r_max = np.max(r)
print(r_max)

bins = np.linspace(0, r_max, 7)
mid = 0.5 * (bins[1:] + bins[:-1])

e_t_mean, _, _ = binned_statistic(r[sel_bg], e_t[sel_bg], bins=bins)
e_x_mean, _, _ = binned_statistic(r[sel_bg], e_x[sel_bg], bins=bins)

e_t_std, _, _ = binned_statistic(r[sel_bg], e_t[sel_bg], bins=bins,
                                 statistic='std')
e_x_std, _, _ = binned_statistic(r[sel_bg], e_x[sel_bg], bins=bins,
                                 statistic='std')

count, _, _ = binned_statistic(r[sel_bg], e_t[sel_bg], bins=bins,
                               statistic='count')

e_t_err = e_t_std / count ** 0.5
e_x_err = e_x_std / count ** 0.5

plt.figure()
plt.errorbar(mid, e_t_mean, e_t_err, fmt='o', label='e_t')
plt.errorbar(mid+r_max*0.03, e_x_mean, e_x_err, fmt='x', label='e_x')
plt.axhline(0, c='k', ls=':')
plt.xlabel('r [arcmin]')
plt.ylabel('<e_i>')

plt.legend()

> Figure 7: Mean shape profile with error bars estimated by statistical uncertainty.

Also use bootstrapping to estimate the error bar. Very close result to the previous one.

In [ ]:
def get_stat(data, statistic=np.mean, confidence_level=0.682):
    rng = np.random.default_rng()
    res = bootstrap((data,),
                    statistic=statistic,
                    confidence_level=confidence_level,
                    random_state=rng)
    stat = np.mean(data)
    lerr = stat - res.confidence_interval[0]
    herr = res.confidence_interval[1] - stat
    return stat, lerr, herr, res.standard_error

In [ ]:
inds = np.digitize(r[sel_bg], bins)

e_t_mean_list = []
e_t_lerr_list = []
e_t_herr_list = []
e_t_std_list = []

e_x_mean_list = []
e_x_lerr_list = []
e_x_herr_list = []
e_x_std_list = []

for i in range(1, len(bins)):
    sel = inds == i
    mean, lerr, herr, std = get_stat(np.ma.getdata(e_t[sel_bg][sel]))
    e_t_mean_list.append(mean)
    e_t_lerr_list.append(lerr)
    e_t_herr_list.append(herr)
    e_t_std_list.append(std)

    mean, lerr, herr, std = get_stat(np.ma.getdata(e_x[sel_bg][sel]))
    e_x_mean_list.append(mean)
    e_x_lerr_list.append(lerr)
    e_x_herr_list.append(herr)
    e_x_std_list.append(std)


In [ ]:
plt.figure()
plt.errorbar(mid, e_t_mean_list, [e_t_lerr_list, e_t_herr_list],
             fmt='o', label='e_t')
plt.errorbar(mid+r_max*0.03, e_x_mean_list, [e_x_lerr_list, e_x_herr_list],
             fmt='x', label='e_x')
plt.axhline(0, c='k', ls=':')
plt.xlabel('r [arcmin]')
plt.ylabel('<e_i>')
plt.legend()

> Figure 8: Mean shape profile with error bars estimated by bootstrapping.

##  5. Exercises for the learner
1. Use a big cutout (notebook 03c) to get the coadd image.
2. Test the shape distribution of cluster member galaxies.
3. Test other clusters.